# Abfrage gegen das Digitale Personenregister

*2022-02-10* Quelle: Domherren, GS_Domherren_DB.accdb = MySQL-Datenbank `domherr`

In [1]:
cd("C:/Users/georg/Documents/projekte/GSquery")

In [2]:
using Pkg

In [3]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\GSquery`


In [4]:
using MySQL, DataFrames

In [5]:
using Revise

In [6]:
using GSquery

Funktionen zur Interaktion mit einer Datenbank

In [7]:
dbwiag = nothing

function setDBWIAG(;pwd = missing, host = "127.0.0.1", user = "wiag", db = "wiag")
    global dbwiag
    if !isnothing(dbwiag)
        DBInterface.close!(dbwiag)
    end

    if ismissing(pwd)
        io_pwd = Base.getpass("Passwort für User " * user)
        pwd = readline(io_pwd)
        Base.shred!(io_pwd)
    end
    dbwiag = DBInterface.connect(MySQL.Connection, host, user, pwd, db = db)
end

"""
    sql_df(sql)

execute the commands in `sql` and return a DataFrame
"""
sql_df(sql) = DBInterface.execute(dbwiag, sql) |> DataFrame

sql_df

In [8]:
setDBWIAG(user = "georg", db = "domherr")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="domherr")

In [9]:
table_name = "tbl_domherren"
sql = "SELECT ID_Domherr, Vorname, Praefix, Familienname, Sterbedatum, Geburtsdatum, GSN_ID, GND_ID
FROM $(table_name)
WHERE Status in ('fertig', 'online')";
df_cn = sql_df(sql);

In [10]:
size(df_cn)

(6161, 8)

In [11]:
table_name = "tbl_domherren_aemter";
table_occ_norm = "tbl_aemterliste_normiert";
sql = "SELECT ID_Domherr, Amtsart AS Amtsart_orig, occn.AmtNorm AS Amtsart 
FROM $(table_name) AS occ 
JOIN $(table_occ_norm) AS occn ON occ.ID_Amt_norm = occn.id_amt_norm";
dfocc = sql_df(sql);

In [12]:
GSquery.setcolnameid("ID_Domherr")

:ID_Domherr

In [13]:
GSquery.setminmatchkey()

GSquery.QRecord(Dict{String, Any}("zeitguete" => 0, "muster" => "vn ae ab ao"))

In [46]:
GSquery.Rank.setdrank()
GSquery.Rank.ranklist()[55:65]

11-element Vector{Pair{String, Int64}}:
 "sd vn ao at" => 55
 "sd ae ab ao" => 56
 "sd ae ab at" => 57
 "sd ae ao at" => 58
 "sd ab ao at" => 59
 "vn ae ab ao" => 60
 "vn ae ab at" => 61
 "vn ae ao at" => 62
 "vn ab ao at" => 63
 "ae ab ao at" => 64
    "fn sd vn" => 65

Sortiere die Muster so, dass Kombinationen aus Name und Zeitstellung hoch bewertet werden.

In [40]:
rl = GSquery.Rank.ranklist()
df_rank = DataFrame(rank = [p.first for p in rl], key = [p.second for p in rl]);

In [42]:
filename = "C:\\Users\\georg\\Documents\\projekte-doc\\GS-Reconcile\\Rangfolge-Muster.csv"
CSV.write(filename, df_rank)

"C:\\Users\\georg\\Documents\\projekte-doc\\GS-Reconcile\\Rangfolge-Muster.csv"

In [54]:
filename = "C:\\Users\\georg\\Documents\\projekte-doc\\GS-Reconcile\\Rangfolge-Muster-angepasst.csv"
GSquery.Rank.setdrank(filename)

┌ Info: Lies Übereinstimmungsmuster aus 
│   fileranks = C:\Users\georg\Documents\projekte-doc\GS-Reconcile\Rangfolge-Muster-angepasst.csv
└ @ GSquery.Rank C:\Users\georg\Documents\projekte\GSquery\src\GSquery.jl:234


In [57]:
GSquery.Rank.ranklist()[80:90]

11-element Vector{Pair{String, Int64}}:
          "fn ae" => 80
          "fn ab" => 81
          "sd vn" => 82
          "vn ae" => 83
          "vn ab" => 84
       "fn vn ao" => 85
       "fn vn at" => 86
       "fn ao at" => 87
 "sd ae ab ao at" => 88
    "sd ae ab ao" => 89
    "sd ae ab at" => 90

In [58]:
GSquery.setminmatchkey("vn ab")

GSquery.QRecord(Dict{String, Any}("zeitguete" => 0, "muster" => "vn ab"))

In [59]:
GSquery.Rank.getrank("vn ab")

84

Schließe keine Ämter von dem Vergleich aus.

In [60]:
GSquery.GSOcc.setoccupations(String[])

String[]

Die Ämter sind alle einem Amt in `tbl_aemterliste_normiert` zugeordnet, damit ist ein Mapping für die Abfrage mit `GSOcc.setequivalentoccupations` nicht nötig.

Abfrage

In [61]:
df_gs = GSquery.makeGSDataFrame(df_cn);

In [34]:
names(df_gs);

Auswahl für einen Test

In [15]:
using Random

In [16]:
idx = randsubseq(1:size(df_gs, 1), 0.015);

In [17]:
size(idx)

(96,)

In [62]:
df_gs_t1 = df_gs[idx, :];

In [19]:
querycols = [:Vorname, :Familienname]

2-element Vector{Symbol}:
 :Vorname
 :Familienname

In [63]:
GSquery.reconcile!(df_gs_t1, querycols, dfocc; nmsg = 20, toldateofdeath = 2, toloccupation = 2)

Datensatz: 20
Datensatz: 40
Datensatz: 60
Datensatz: 80


┌ Info: Level:
│   minkey = vn ab
│   minrank = 84
└ @ GSquery C:\Users\georg\Documents\projekte\GSquery\src\GSquery.jl:454
┌ Info: Gültige Ämter
│   GSOcc.occupations = String[]
└ @ GSquery C:\Users\georg\Documents\projekte\GSquery\src\GSquery.jl:455
┌ Info: Abfragespalten
│   querycols = [:Vorname, :Familienname]
└ @ GSquery C:\Users\georg\Documents\projekte\GSquery\src\GSquery.jl:456


2-element Vector{Pair{Int64, Int64}}:
 0 => 91
 1 => 5

In [22]:
using CSV

In [64]:
filename = "C:\\Users\\georg\\Documents\\projekte-doc\\GS-Reconcile\\Domherren-Abgleich-GS.csv"

"C:\\Users\\georg\\Documents\\projekte-doc\\GS-Reconcile\\Domherren-Abgleich-GS.csv"

In [65]:
CSV.write(filename, df_gs_t1)

"C:\\Users\\georg\\Documents\\projekte-doc\\GS-Reconcile\\Domherren-Abgleich-GS.csv"